In [2]:
import pandas as pd
import os
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import clear_output
import plotly.graph_objects as go

In [3]:
TICKERS_CSV = "data/averages/average_hot_scores.csv"  
OUTPUT_DIR = "output/cash_flow"  

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:

df_tickers = pd.read_csv(TICKERS_CSV)

tickers = (
    df_tickers["symbol"]
    .dropna()
    .astype(str)
    .unique()
    .tolist()
)



In [ ]:
for ticker in tickers:
    try:
        t = yf.Ticker(ticker)
        cf = t.quarterly_cashflow.T  # transpose for easier plotting

        if cf.empty:
            print(f"{ticker}: No cash flow data")
            continue

        # Convert index to datetime
        cf.index = pd.to_datetime(cf.index)

        # Create Plotly figure
        fig = go.Figure()

        # Add line per cash flow component
        for col in columns_to_plot:
            if col in cf.columns:
                fig.add_trace(go.Scatter(
                    x=cf.index,
                    y=cf[col],
                    mode="lines+markers",
                    name=col,
                    line=dict(width=2),
                    marker=dict(size=6),
                    hovertemplate='<b>%{x|%Y-%m-%d}</b><br>%{y:$,.0f}<extra></extra>'
                ))

        # Layout
        fig.update_layout(
            title=f"{ticker} — Quarterly Cash Flow Components",
            xaxis_title="Quarter End Date",
            yaxis_title="Cash Flow (USD)",
            template="plotly_dark",
            hovermode="x unified",
            margin=dict(l=80, r=40, t=80, b=60),
            height=500,
            font=dict(size=13),
            legend=dict(
                x=-0.15,
                y=1,
                xanchor="left",
                yanchor="top"
            )
        )

        chart_path = os.path.join(OUTPUT_DIR, f"{ticker}.html")
        fig.write_html(chart_path, include_plotlyjs="cdn")

    except Exception as e:
        print(f"{ticker}: error → {e}")